## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import os
import gmaps
import numpy as np
#Import API key
from config import g_key

#Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file.
file_to_read = os.path.join("..", "Weather_Database", "WeatherPy_Database.csv")
city_data_df = pd.read_csv(file_to_read)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Guerrero Negro,MX,27.9769,-114.0611,54.10,35,0,0.72,clear sky
1,1,Vostok,RU,46.4856,135.8833,-12.28,100,100,3.51,overcast clouds
2,2,Bubaque,GW,11.2833,-15.8333,79.43,56,0,10.13,clear sky
3,3,Hermanus,ZA,-34.4187,19.2345,76.89,53,7,23.02,clear sky
4,4,Barrow,US,71.2906,-156.7887,-29.18,68,0,6.91,clear sky


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
validator = 0
dfmin = city_data_df["Max Temp"].min()
dfmax = city_data_df["Max Temp"].max()
while validator == 0:
    mint = int(input("What is your minimum temperature preference: "))
    if mint < dfmin:
        print("Minimum too low")
    elif mint > dfmax:
        print("Minimum too high")
    else:
        validator = 1
while validator == 1:
    maxt = int(input("What is your maximum temperature preference: "))
    if maxt < dfmin:
        print("Maximum too low")
    elif maxt < mint:
        print("Maximum too low")
    elif maxt > dfmax:
        print("Maximum too High")
    else:
        validator = 2

What is your minimum temperature preference: 40
What is your maximum temperature preference: 70


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filter_df = city_data_df.loc[(city_data_df['Max Temp'] > mint) & (city_data_df['Max Temp'] < maxt)]
filter_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Guerrero Negro,MX,27.9769,-114.0611,54.10,35,0,0.72,clear sky
7,7,Beyneu,KZ,45.3167,55.2000,40.42,73,100,19.17,overcast clouds
8,8,Bluff,NZ,-46.6000,168.3333,48.79,66,66,10.80,broken clouds
9,9,Hit,IQ,33.6416,42.8251,56.10,56,42,10.00,scattered clouds
10,10,Rusera,IN,25.7500,86.0333,56.64,91,100,6.80,overcast clouds
12,12,Los Llanos De Aridane,ES,28.6585,-17.9182,67.12,66,20,5.99,few clouds
18,18,Half Moon Bay,US,37.4636,-122.4286,52.61,88,0,0.00,clear sky
19,19,Hobart,AU,-42.8794,147.3294,59.25,83,100,5.75,overcast clouds
21,21,Kapaa,US,22.0752,-159.3190,66.18,94,0,5.75,clear sky
23,23,Cabo San Lucas,MX,22.8909,-109.9124,68.50,54,0,0.00,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
filter_df.isnull().sum()
#no empty rows, but empty columns are present

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
filter_df.dropna(inplace = True)
clean_df = filter_df.copy()
clean_df.isnull().sum()

C:\Users\miste\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Guerrero Negro,MX,54.10,clear sky,27.9769,-114.0611,
7,Beyneu,KZ,40.42,overcast clouds,45.3167,55.2000,
8,Bluff,NZ,48.79,broken clouds,-46.6000,168.3333,
9,Hit,IQ,56.10,scattered clouds,33.6416,42.8251,
10,Rusera,IN,56.64,overcast clouds,25.7500,86.0333,
12,Los Llanos De Aridane,ES,67.12,few clouds,28.6585,-17.9182,
18,Half Moon Bay,US,52.61,clear sky,37.4636,-122.4286,
19,Hobart,AU,59.25,overcast clouds,-42.8794,147.3294,
21,Kapaa,US,66.18,clear sky,22.0752,-159.3190,
23,Cabo San Lucas,MX,68.50,clear sky,22.8909,-109.9124,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Guerrero Negro,MX,54.10,clear sky,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"
7,Beyneu,KZ,40.42,overcast clouds,45.3167,55.2000,NUR
8,Bluff,NZ,48.79,broken clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
9,Hit,IQ,56.10,scattered clouds,33.6416,42.8251,صالون زيرو
10,Rusera,IN,56.64,overcast clouds,25.7500,86.0333,Santosh Hotel&Santosh resturent
12,Los Llanos De Aridane,ES,67.12,few clouds,28.6585,-17.9182,Valle Aridane
18,Half Moon Bay,US,52.61,clear sky,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
19,Hobart,AU,59.25,overcast clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
21,Kapaa,US,66.18,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
23,Cabo San Lucas,MX,68.50,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos


In [12]:
#check for nulls again
hotel_df.isnull().sum()

City                    0
Country                 0
Max Temp                0
Current Description     0
Lat                     0
Lng                     0
Hotel Name             14
dtype: int64

In [13]:
#yeet the nulls
clean_hotel_df = hotel_df.copy()

clean_hotel_df.dropna(inplace = True)
clean_hotel_df.isnull().sum()


City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [14]:
# 8a. Create the output File (CSV)
output_data_file = os.path.join("..", "Vacation_Search", "WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))